In [11]:
from pandas import pandas as pd
import datetime
import hashlib 
import os
from lxml import etree as et
from itertools import islice
from datetime import datetime
import datetime as dt
import time
import glob

In [12]:
def flip_function(df):
    flip_data = df.T
    flip_data.columns=flip_data.iloc[0]
    flip_data.reset_index()
    flip_data=flip_data[1:]
    return flip_data

def information_events_utc(df, start_date_time):
#     '''
#     Parameters:
#     df: dataframe
#         dataframe with a column 'StartDateTimeUTC' that is in yyyy-mm-dd format that you want in utc
#     start_date_time: string
#         the start date time frome the project sheet

#     Returns:
#     A formatted dataframe
#     '''


    date = start_date_time.split('T')[0]
    z_thing = '.000000Z'
    previous_hour = 0

    # loop over the dataframe
    for index, row in df.iterrows():

        # #ignore the first and last row
        # if (index == 0) or (index==len(df)-1):
        #     continue


        # check and see if the time is allready formatted
        start_time = row['StartDateTime']

        try:
            if (len(str(start_time))) > 10 or (str(start_time)=='na'):
                continue
        except:
            continue



        # find the hour of the event
        current_hour = str(row['StartDateTime'])[:2]
        # print(current_hour)

        # if the current hour is less than the previous hour, we know we crossed midnight, so increment the day
        if int(current_hour) < int(previous_hour):
            datetime_obj = datetime.strptime(date, '%Y-%m-%d')

            datetime_obj += dt.timedelta(days=1)
            date = str(datetime_obj).split(' ')[0]

        else:
            pass

        parsed_event_start_time = date+'T'+str(start_time)+z_thing
        # print(parsed_event_time)

        # update hour
        previous_hour = current_hour

        # update df
        df.at[index, 'StartDateTime'] = parsed_event_start_time

    if 'EndDateTime' in df.columns:
        date = start_date_time.split('T')[0]
        z_thing = '.000000Z'
        previous_hour = 0


        for index, row in df.iterrows():

            # #ignore the first and last row
            # if (index == 0) or (index==len(df)-1):
            #     continue


            # check and see if the time is allready formatted
            end_time = row['EndDateTime']

            try:
                if (len(str(end_time))) > 10 or (str(end_time)=='nan'):
                    continue

            except:
                continue

            # find the hour of the event
            current_hour = str(row['EndDateTime'])[:2]
            # print(current_hour)

            # if the current hour is less than the previous hour, we know we crossed midnight
            if int(current_hour) < int(previous_hour):
                datetime_obj = datetime.strptime(date, '%Y-%m-%d')

                datetime_obj += dt.timedelta(days=1)
                date = str(datetime_obj).split(' ')[0]
                # print('adding_date')

            else:
                pass

            parsed_event_end_time = date+'T'+str(end_time)+z_thing
            # print(parsed_event_time)

            # update hour
            previous_hour = current_hour

            # update df
            df.at[index, 'EndDateTime'] = parsed_event_end_time

    return df

In [29]:
unprocessed_spreadsheets = 'C:/Users/jmdoherty/OneDrive - University of Massachusetts/Desktop/direc'
p_direc= 'C:/Users/jmdoherty/OneDrive - University of Massachusetts/Desktop/p_direc'


unprocessed_spreadsheets = glob.glob(os.path.join(unprocessed_spreadsheets, '*.xlsx'))
# ------------------------------------------------------------------------
# STARTED CODE FOR CHECKING FOLDERS FOR MATCHING FILES. NEED TO FIX....

# for i in unprocessed_spreadsheets:
#     for f in p_direc:
#         if os.path.basename(i.replace('.xlsx','.xml')).isfile(f)== True:
#             break
#         else: continue
# -----------------------------------------------------------------------            
#         if i.endswith('.xlsx'):
#             pass
#         else:

#             input_filename = i.split('.')[0]+'.xlsx'
#             print(i, input_filename)

#     print(i)

    parse_list = i.split('_')
#     print(parse_list)

    sensor = parse_list[2]
    level = parse_list[3]
    
#     print(sensor, level)
            
            
                 
    if sensor == 'GNSS' and level== 'L0':
#             xlsx = []
#             for i in os.listdir(unprocessed_spreadsheets):
#                     if i.endswith('.xlsx'):
#                         i_split = i.split('.')[0]
#                         list = xlsx.append(i_split)
#                     else:
#                         continue

#             for i in range(len(xlsx)):

        f = i
        sbf_f= i.replace('.xlsx', '.22_')


        project_prep = pd.read_excel(f, sheet_name='Project')
        projectdf= pd.DataFrame(project_prep)
        CollectionMetaData=flip_function(projectdf)
        auto_start_date_time= CollectionMetaData.iloc[0,8]

        SurfaceTypeCodes=pd.read_excel(f, sheet_name='SurfaceTypeCodes')
        MotionCodes=pd.read_excel(f, sheet_name='MotionCodes')
        Events = pd.read_excel(f, sheet_name='Events')
        EventsUTC=information_events_utc(Events, auto_start_date_time)
        InformationEvents = pd.read_excel(f, sheet_name='InformationEvents')
        InformationEventsUTC= information_events_utc(InformationEvents, auto_start_date_time)
        AntennaOffsets= pd.read_excel(f, sheet_name='AntennaOffsets')

        md5_hash=hashlib.md5()
        file=open(sbf_f,"rb")
        content=file.read()
        md5_hash.update(content)
        digest=md5_hash.hexdigest()

        file_size = os.path.getsize(sbf_f)

        ti_m=os.path.getmtime(sbf_f)
        m_ti=time.ctime(ti_m)
        t_obj=time.strptime(m_ti)
        T_stamp=time.strftime("%Y-%m-%dT%H:%M:%S.000000Z",t_obj)

        current_time= str(datetime.now()).replace(" ","T")+("Z")

        for index, row in AntennaOffsets.iterrows():
            for j, r in InformationEvents.iterrows():
                if row["StartDateTime"]==r["StartDateTime"]:
                    InformationEvents.at[j,"Description"]= row["All"]

        root= et.Element('GranuleMetaDataFile')

        for index,row in CollectionMetaData.iterrows():
            Column_heading_1 = et.SubElement(root,'Project')
            Column_heading_2 = et.SubElement(root, 'FileIdentifier')

        root_tags=et.SubElement(root,'CollectionMetaData')

        for index,row in CollectionMetaData.iterrows():
            Column_heading_3 = et.SubElement(root_tags, 'Campaign')
            Column_heading_4 = et.SubElement(root_tags, 'CampaignShortName')
            Column_heading_5 = et.SubElement(root_tags, 'ShortName')
            Column_heading_6 = et.SubElement(root_tags, 'InstrumentName')
            Column_heading_7 = et.SubElement(root_tags, 'Description')

            Column_heading_1.text = str(row['Project'])
            Column_heading_2.text = str(row['FileIdentifier'])
            Column_heading_3.text = str(row['Campaign'])
            Column_heading_4.text = str(row['CampaignShortName'])
            Column_heading_5.text = str(row['ShortName'])
            Column_heading_6.text = str(row['InstrumentName'])
            Column_heading_7.text = str(row['Description'])


        root_tags = et.SubElement(root, 'Extent')
        root_tags1 = et.SubElement(root_tags,'TemporalExtent')
        root_tags2 = et.SubElement(root_tags,'SpatialExtent')

        for index,row in CollectionMetaData.iterrows():
            Column_heading_8 = et.SubElement(root_tags1, 'StartDateTime')
            Column_heading_9 = et.SubElement(root_tags1, 'EndDateTime')
            Column_heading_10 = et.SubElement(root_tags2, 'WestBoundLongitude')
            Column_heading_11 = et.SubElement(root_tags2, 'EastBoundLongitude')
            Column_heading_12 = et.SubElement(root_tags2, 'SouthBoundLatitude')
            Column_heading_13 = et.SubElement(root_tags2, 'NorthBoundLatitude')

            Column_heading_8.text = str(row['StartDateTime'])
            Column_heading_9.text = str(row['EndDateTime'])
            Column_heading_10.text = str(row['WestBoundLongitude'])
            Column_heading_11.text = str(row['EastBoundLongitude'])
            Column_heading_12.text = str(row['SouthBoundLatitude'])
            Column_heading_13.text = str(row['NorthBoundLatitude'])

        root_tags = et.SubElement(root, 'DataGranuleMembers')

        for index,row in CollectionMetaData.iterrows():
            root_tags1= et.SubElement(root_tags,'DataGranuleMember')
            Column_heading_14 = et.SubElement(root_tags1, 'LocalGranuleID')
            Column_heading_15 = et.SubElement(root_tags1, 'ProductionDateTime')

            Column_heading_14.text = str(row['LocalGranuleID'])
            Column_heading_15.text = str(current_time)

        for index,row in CollectionMetaData.iterrows():
            root_tags1= et.SubElement(root_tags,'DataGranuleMember')
            Column_heading_16 = et.SubElement(root_tags1, 'LocalGranuleID')
            Column_heading_17 = et.SubElement(root_tags1, 'ProductionDateTime')
            Column_heading_18 = et.SubElement(root_tags1, 'Checksum')
            Column_heading_19 = et.SubElement(root_tags1, 'ChecksumType')
            Column_heading_20 = et.SubElement(root_tags1, 'Size')

            Column_heading_16.text = str(row['LocalGranuleID_SBF'])
            Column_heading_17.text = str(T_stamp)
            Column_heading_18.text = str(digest)
            Column_heading_19.text = str(row['ChecksumType'])
            Column_heading_20.text = str(file_size)

        root_tags = et.SubElement(root, 'Additional')

        for index,row in CollectionMetaData.iterrows():
            Column_heading_21 = et.SubElement(root_tags, 'Personnel')
            Column_heading_22 = et.SubElement(root_tags, 'Contact')
            Column_heading_23 = et.SubElement(root_tags, 'ReceiverModel')
            Column_heading_24 = et.SubElement(root_tags, 'ReceiverSerial')
            Column_heading_25 = et.SubElement(root_tags, 'AntennaModel')
            Column_heading_26 = et.SubElement(root_tags, 'IGSCalibration')
            Column_heading_27 = et.SubElement(root_tags, 'Radome')
            Column_heading_28 = et.SubElement(root_tags, 'AntennaSerial')
            Column_heading_29 = et.SubElement(root_tags, 'AntennaMeasurementPointType')

            Column_heading_21.text = str(row['Personnel'])
            Column_heading_22.text = str(row['Contact'])
            Column_heading_23.text = str(row['ReceiverModel'])
            Column_heading_24.text = str(row['ReceiverSerial'])
            Column_heading_25.text = str(row['AntennaModel'])
            Column_heading_26.text = str(row['IGSCalibration'])
            Column_heading_27.text = str(row['Radome'])
            Column_heading_28.text = str(row['AntennaSerial'])
            Column_heading_29.text = str(row['AntennaMeasurementPointType'])

        root_tags2=et.SubElement(root_tags,'SurfaceTypeCodes')

        for index, row in SurfaceTypeCodes.iterrows():
            root_tags3=et.SubElement(root_tags2,'SurfaceTypeCode')
            Column_heading_30 = et.SubElement(root_tags3,'Code')
            Column_heading_31 = et.SubElement(root_tags3,'Description')

            Column_heading_30.text= str(row['SurfaceTypeCode'])
            Column_heading_31.text= str(row['Description'])

        root_tags1=et.SubElement(root_tags,'MotionCodes')

        for index, row in MotionCodes.iterrows():
            root_tags3=et.SubElement(root_tags1,'MotionCode')
            Column_heading_32 = et.SubElement(root_tags3,'Code')
            Column_heading_33 = et.SubElement(root_tags3,'Description')

            Column_heading_32.text= str(row['MotionCode'])
            Column_heading_33.text= str(row['Description'])

        root_tags1=et.SubElement(root_tags,'Events')

        for index,row in EventsUTC.iterrows():
            root_tags2 = et.SubElement(root_tags1, 'Event')

            Column_heading_34 = et.SubElement(root_tags2, 'StartDateTime')
            Column_heading_35 = et.SubElement(root_tags2, 'SurfaceTypeCode')
            Column_heading_36 = et.SubElement(root_tags2, 'MotionCode')
            Column_heading_37 = et.SubElement(root_tags2, 'AntennaOffset')

            Column_heading_34.text = str(row['StartDateTime'])
            Column_heading_35.text = str(row['SurfaceTypeCode'])
            Column_heading_36.text = str(row['MotionCode'])
            Column_heading_37.text = str(row['AntennaOffset'])

        root_tags1=et.SubElement(root_tags,'InformationEvents')

        for index,row in InformationEventsUTC.iterrows():
            root_tags2= et.SubElement(root_tags1, 'InformationEvent')

            Column_heading_38 = et.SubElement(root_tags2, 'StartDateTime')
            Column_heading_39 = et.SubElement(root_tags2, 'EndDateTime')
            Column_heading_40 = et.SubElement(root_tags2, 'Description')

            Column_heading_38.text = str(row['StartDateTime'])
            Column_heading_39.text = str(row['EndDateTime'])
            Column_heading_40.text = str(row['Description'])

    #     FileName = Column_heading_2.text

        tree = et.ElementTree(root) 
        et.indent(tree, space="\t", level=0)
        tree.write(p_direc+'/'+os.path.basename(i.replace('.xlsx','.xml')), encoding="utf-8")
        

    if sensor == 'ADCP' and level== 'L0':
        f = i
        sbf_f= i.replace('.xlsx', '.tar.gz')
    
    
    
    
    
#     xlsx = []
#     for i in os.listdir(unprocessed_spreadsheets):
#             if i.endswith('.xlsx'):
#                 i_split = i.split('.')[0]
#                 list = xlsx.append(i_split)
#             else:
#                 continue

#     for i in range(len(xlsx)):
#         f = direc+'/'+xlsx[i] +'.xlsx'
#         sbf_f= direc+'/'+xlsx[i] + '.tar.gz'

        project_prep = pd.read_excel(f, sheet_name='Project')
        projectdf= pd.DataFrame(project_prep)
        CollectionMetaData=flip_function(projectdf)
        TransectTypeCodes=pd.read_excel(f, sheet_name='TransectTypeCodes')
        auto_start_date_time= CollectionMetaData.iloc[0,7]
        Events = pd.read_excel(f, sheet_name='Events')
        EventsUTC=information_events_utc(Events, auto_start_date_time)

        md5_hash=hashlib.md5()
        file=open(sbf_f,"rb")
        content=file.read()
        md5_hash.update(content)
        digest=md5_hash.hexdigest()

        file_size = os.path.getsize(sbf_f)

        ti_m=os.path.getmtime(sbf_f)
        m_ti=time.ctime(ti_m)
        t_obj=time.strptime(m_ti)
        T_stamp=time.strftime("%Y-%m-%dT%H:%M:%S.000000Z",t_obj)

        current_time= str(datetime.now()).replace(" ","T")+("Z")

        root= et.Element('GranuleMetaDataFile')

        for index,row in CollectionMetaData.iterrows():
            Column_heading_1 = et.SubElement(root,'Project')
            Column_heading_2 = et.SubElement(root, 'FileIdentifier')

        root_tags=et.SubElement(root,'CollectionMetaData')

        for index,row in CollectionMetaData.iterrows():
            Column_heading_3 = et.SubElement(root_tags, 'Campaign')
            Column_heading_4 = et.SubElement(root_tags, 'CampaignShortName')
            Column_heading_5 = et.SubElement(root_tags, 'ShortName')
            Column_heading_6 = et.SubElement(root_tags, 'InstrumentName')
            Column_heading_7 = et.SubElement(root_tags, 'Description')

            Column_heading_1.text = str(row['Project'])
            Column_heading_2.text = str(row['FileIdentifier'])
            Column_heading_3.text = str(row['Campaign'])
            Column_heading_4.text = str(row['CampaignShortName'])
            Column_heading_5.text = str(row['ShortName'])
            Column_heading_6.text = str(row['InstrumentName'])
            Column_heading_7.text = str(row['Description'])

        root_tags = et.SubElement(root, 'Extent')
        root_tags1 = et.SubElement(root_tags,'TemporalExtent')
        root_tags2 = et.SubElement(root_tags,'SpatialExtent')

        for index,row in CollectionMetaData.iterrows():
            Column_heading_8 = et.SubElement(root_tags1, 'StartDateTime')
            Column_heading_9 = et.SubElement(root_tags1, 'EndDateTime')
            Column_heading_10 = et.SubElement(root_tags2, 'WestBoundLongitude')
            Column_heading_11 = et.SubElement(root_tags2, 'EastBoundLongitude')
            Column_heading_12 = et.SubElement(root_tags2, 'SouthBoundLatitude')
            Column_heading_13 = et.SubElement(root_tags2, 'NorthBoundLatitude')

            Column_heading_8.text = str(row['StartDateTime'])
            Column_heading_9.text = str(row['EndDateTime'])
            Column_heading_10.text = str(row['WestBoundLongitude'])
            Column_heading_11.text = str(row['EastBoundLongitude'])
            Column_heading_12.text = str(row['SouthBoundLatitude'])
            Column_heading_13.text = str(row['NorthBoundLatitude'])

        root_tags = et.SubElement(root, 'DataGranuleMembers')

        for index,row in CollectionMetaData.iterrows():
            root_tags1= et.SubElement(root_tags,'DataGranuleMember')
            Column_heading_14 = et.SubElement(root_tags1, 'LocalGranuleID')
            Column_heading_15 = et.SubElement(root_tags1, 'ProductionDateTime')

            Column_heading_14.text = str(row['LocalGranuleID'])
            Column_heading_15.text = str(current_time)

        for index,row in CollectionMetaData.iterrows():
            root_tags1= et.SubElement(root_tags,'DataGranuleMember')
            Column_heading_16 = et.SubElement(root_tags1, 'LocalGranuleID')
            Column_heading_17= et.SubElement(root_tags1, 'ProductionDateTime')
            Column_heading_18 = et.SubElement(root_tags1, 'Checksum')
            Column_heading_19 = et.SubElement(root_tags1, 'ChecksumType')
            Column_heading_20 = et.SubElement(root_tags1, 'Size')

            Column_heading_16.text = str(row['LocalGranuleID'])
            Column_heading_17.text = str(T_stamp)
            Column_heading_18.text = str(digest)
            Column_heading_19.text = str(row['ChecksumType'])
            Column_heading_20.text = str(file_size)

        root_tags = et.SubElement(root, 'Additional')

        for index,row in CollectionMetaData.iterrows():
            Column_heading_21 = et.SubElement(root_tags, 'Personnel')
            Column_heading_22 = et.SubElement(root_tags, 'Contact')
            Column_heading_23 = et.SubElement(root_tags, 'ADCPModel')
            Column_heading_24 = et.SubElement(root_tags, 'ADCPSerial')
            Column_heading_25 = et.SubElement(root_tags, 'AntennaModel')
            Column_heading_26 = et.SubElement(root_tags, 'IGSCalibration')
            Column_heading_27 = et.SubElement(root_tags, 'Radome')
            Column_heading_28 = et.SubElement(root_tags, 'AntennaSerial')
            Column_heading_29 = et.SubElement(root_tags, 'TransducerDepth_m')
            Column_heading_30 = et.SubElement(root_tags, 'MagneticVariation')
            Column_heading_31 = et.SubElement(root_tags, 'Positionmethod')
            Column_heading_32 = et.SubElement(root_tags, 'TransectLocation')

            Column_heading_21.text = str(row['Personnel'])
            Column_heading_22.text = str(row['Contact'])
            Column_heading_23.text = str(row['ADCPModel'])
            Column_heading_24.text = str(row['ADCPSerial'])
            Column_heading_25.text = str(row['AntennaModel'])
            Column_heading_26.text = str(row['IGSCalibration'])
            Column_heading_27.text = str(row['Radome'])
            Column_heading_28.text = str(row['AntennaSerial'])
            Column_heading_29.text = str(row['TransducerDepth_m'])
            Column_heading_30.text = str(row['MagneticVariation'])
            Column_heading_31.text = str(row['Positionmethod'])
            Column_heading_32.text = str(row['TransectLocation'])

        root_tags2=et.SubElement(root_tags,'TransectTypes')

        for index, row in TransectTypeCodes.iterrows():
            root_tags3=et.SubElement(root_tags2,'TransectType')
            Column_heading_33 = et.SubElement(root_tags3,'TransectTypeCode')
            Column_heading_34 = et.SubElement(root_tags3,'Description')

            Column_heading_33.text= str(row['TransectTypeCode'])
            Column_heading_34.text= str(row['Description'])

        root_tags1=et.SubElement(root_tags,'Events')

        for index,row in EventsUTC.iterrows():
            root_tags2 = et.SubElement(root_tags1, 'Event')

            Column_heading_35 = et.SubElement(root_tags2, 'StartDateTime')
            Column_heading_36 = et.SubElement(root_tags2, 'TransectFile')
            Column_heading_37 = et.SubElement(root_tags2, 'TransectStart')
            Column_heading_38 = et.SubElement(root_tags2, 'TransectEnd')
            Column_heading_39 = et.SubElement(root_tags2, 'TransectType')

            Column_heading_35.text = str(row['StartDateTime'])
            Column_heading_36.text = str(row['TransectFile'])
            Column_heading_37.text = str(row['TransectStart'])
            Column_heading_38.text = str(row['TransectEnd'])
            Column_heading_39.text = str(row['TransectType'])

        tree = et.ElementTree(root) 
        et.indent(tree, space="\t", level=0)
        tree.write(p_direc+'/'+os.path.basename(i.replace('.xlsx','.xml')), encoding="utf-8")
    
    if sensor == 'BL' and level== 'L0':
        f = i
        xle_f= i.replace('.xlsx', '.xle')
        csv_f= i.replace('.xlsx', '.csv')
        
#         xlsx = []
#         for i in os.listdir(unprocessed_spreadsheets):
#                 if i.endswith('.xlsx'):
#                     i_split = i.split('.')[0]
#                     list = xlsx.append(i_split)
#                 else:
#                     continue

#         for i in range(len(xlsx)):

#             f = direc+'/'+xlsx[i] +'.xlsx'
#             xle_f= direc+'/'+xlsx[i] + '.xle'
#             csv_f= direc+'/'+xlsx[i] + '.csv'

        project_prep = pd.read_excel(f, sheet_name='Project')
        projectdf= pd.DataFrame(project_prep)
        CollectionMetaData=flip_function(projectdf)
        DataGranuleMembers = pd.read_excel(f, sheet_name='DataGranuleMembers')


        md5_hash=hashlib.md5()
        file=open(xle_f,"rb")
        content=file.read()
        md5_hash.update(content)
        xle_digest=md5_hash.hexdigest()

        xle_file_size = os.path.getsize(xle_f)  

        ti_m=os.path.getmtime(xle_f)
        m_ti=time.ctime(ti_m)
        t_obj=time.strptime(m_ti)
        xle_T_stamp=time.strftime("%Y-%m-%dT%H:%M:%S.000000Z",t_obj)

            # print(xle_file_size)

        md5_hash=hashlib.md5()
        file=open(csv_f,"rb")
        content=file.read()
        md5_hash.update(content)
        csv_digest=md5_hash.hexdigest()
        csv_file_size= os.path.getsize(csv_f)

        ti_m=os.path.getmtime(csv_f)
        m_ti=time.ctime(ti_m)
        t_obj=time.strptime(m_ti)
        csv_T_stamp=time.strftime("%Y-%m-%dT%H:%M:%S.000000Z",t_obj)

        #Need to be sure that this is in UTC time and NOT EST. I think it is getting the time based on the time from the local device, might beed to configure something on the cluster?
        current_time= str(datetime.now()).replace(" ","T")+("Z")

        root= et.Element('GranuleMetaDataFile')

        for index,row in CollectionMetaData.iterrows():
            Column_heading_1 = et.SubElement(root,'Project')
            Column_heading_2 = et.SubElement(root, 'FileIdentifier')

        root_tags=et.SubElement(root,'CollectionMetaData')

        for index,row in CollectionMetaData.iterrows():
            Column_heading_3 = et.SubElement(root_tags, 'Campaign')
            Column_heading_4 = et.SubElement(root_tags, 'CampaignShortName')
            Column_heading_5 = et.SubElement(root_tags, 'ShortName')
            Column_heading_6 = et.SubElement(root_tags, 'InstrumentName')
            Column_heading_7 = et.SubElement(root_tags, 'Description')

            Column_heading_1.text = str(row['Project'])
            Column_heading_2.text = str(row['FileIdentifier'])
            Column_heading_3.text = str(row['Campaign'])
            Column_heading_4.text = str(row['CampaignShortName'])
            Column_heading_5.text = str(row['ShortName'])
            Column_heading_6.text = str(row['InstrumentName'])
            Column_heading_7.text = str(row['Description'])

        root_tags = et.SubElement(root, 'Extent')
        root_tags1 = et.SubElement(root_tags,'TemporalExtent')
        root_tags2 = et.SubElement(root_tags,'SpatialExtent')

        for index,row in CollectionMetaData.iterrows():
            Column_heading_8 = et.SubElement(root_tags1, 'StartDateTime')
            Column_heading_9 = et.SubElement(root_tags1, 'EndDateTime')
            Column_heading_10 = et.SubElement(root_tags2, 'WestBoundLongitude')
            Column_heading_11 = et.SubElement(root_tags2, 'EastBoundLongitude')
            Column_heading_12 = et.SubElement(root_tags2, 'SouthBoundLatitude')
            Column_heading_13 = et.SubElement(root_tags2, 'NorthBoundLatitude')

            Column_heading_8.text = str(row['StartDateTime'])
            Column_heading_9.text = str(row['EndDateTime'])
            Column_heading_10.text = str(row['WestBoundLongitude'])
            Column_heading_11.text = str(row['EastBoundLongitude'])
            Column_heading_12.text = str(row['SouthBoundLatitude'])
            Column_heading_13.text = str(row['NorthBoundLatitude'])

        root_tags = et.SubElement(root, 'DataGranuleMembers')

        for index,row in CollectionMetaData.iterrows():
            root_tags1= et.SubElement(root_tags,'DataGranuleMember')
            Column_heading_14 = et.SubElement(root_tags1, 'LocalGranuleID')
            Column_heading_15 = et.SubElement(root_tags1, 'ProductionDateTime')

            Column_heading_14.text = str(row['LocalGranuleID'])
            Column_heading_15.text = str(current_time)

        for index,row in DataGranuleMembers.iterrows():
            if str(".xle") in (row['LocalGranuleID']):
                root_tags1= et.SubElement(root_tags,'DataGranuleMember')
                Column_heading_16 = et.SubElement(root_tags1, 'LocalGranuleID')
                Column_heading_17 = et.SubElement(root_tags1, 'ProductionDateTime')
                Column_heading_18 = et.SubElement(root_tags1, 'Checksum')
                Column_heading_19 = et.SubElement(root_tags1, 'CheckSumType')
                Column_heading_20 = et.SubElement(root_tags1, 'Size')

                Column_heading_16.text = str(row['LocalGranuleID'])
                Column_heading_17.text = str(xle_T_stamp)
                Column_heading_18.text = str(xle_digest)
                Column_heading_19.text = str(row['CheckSumType'])
                Column_heading_20.text = str(xle_file_size)
            else:
                root_tags1= et.SubElement(root_tags,'DataGranuleMember')
                Column_heading_16 = et.SubElement(root_tags1, 'LocalGranuleID')
                Column_heading_17 = et.SubElement(root_tags1, 'ProductionDateTime')
                Column_heading_18 = et.SubElement(root_tags1, 'Checksum')
                Column_heading_19 = et.SubElement(root_tags1, 'CheckSumType')
                Column_heading_20 = et.SubElement(root_tags1, 'Size')

                Column_heading_16.text = str(row['LocalGranuleID'])
                Column_heading_17.text = str(csv_T_stamp)
                Column_heading_18.text = str(csv_digest)
                Column_heading_19.text = str(row['CheckSumType'])
                Column_heading_20.text = str(csv_file_size)

        root_tags = et.SubElement(root, 'Additional')

        for index,row in CollectionMetaData.iterrows():
            Column_heading_21 = et.SubElement(root_tags, 'Personnel')
            Column_heading_22 = et.SubElement(root_tags, 'Contact')
            Column_heading_23 = et.SubElement(root_tags, 'BLModel')
            Column_heading_24 = et.SubElement(root_tags, 'BLSerial')
            Column_heading_25 = et.SubElement(root_tags, 'Latitude')
            Column_heading_26 = et.SubElement(root_tags, 'Longitude')
            Column_heading_27 = et.SubElement(root_tags, 'Install_DateTime')
            Column_heading_28 = et.SubElement(root_tags, 'Uninstall_DateTime')


            Column_heading_21.text = str(row['Personnel'])
            Column_heading_22.text = str(row['Contact'])
            Column_heading_23.text = str(row['BLModel'])
            Column_heading_24.text = str(row['BLSerial'])
            Column_heading_25.text = str(row['Latitude'])
            Column_heading_26.text = str(row['Longitude'])
            Column_heading_27.text = str(row['Install_DateTime'])
            Column_heading_28.text = str(row['Uninstall_DateTime'])


    #     FileName = Column_heading_2.text 

        tree = et.ElementTree(root) 
        et.indent(tree, space="\t", level=0)
        tree.write(p_direc+'/'+os.path.basename(i.replace('.xlsx','.xml')), encoding="utf-8")

    if sensor == 'PT' and level== 'L0':
        f = i
        xle_f= i.replace('.xlsx', '.xle')
        csv_f= i.replace('.xlsx', '.csv')

        
        
        
        
#             xlsx = []
#             for i in os.listdir(unprocessed_spreadsheets):
#                     if i.endswith('.xlsx'):
#                         i_split = i.split('.')[0]
#                         list = xlsx.append(i_split)
#                     else:
#                         continue

#             for i in range(len(xlsx)):

#                 f = direc+xlsx[i] +'.xlsx'
#                 xle_f= direc+xlsx[i] + '.xle'
#                 csv_f= direc+xlsx[i] + '.csv'

                # print(csv_f)


        project_prep = pd.read_excel(f, sheet_name='Project')
        projectdf= pd.DataFrame(project_prep)
        CollectionMetaData=flip_function(projectdf)
        DataGranuleMembers= pd.read_excel(f,sheet_name='DataGranuleMembers')

        md5_hash=hashlib.md5()
        file=open(xle_f,"rb")
        content=file.read()
        md5_hash.update(content)
        xle_digest=md5_hash.hexdigest()

        xle_file_size = os.path.getsize(xle_f)

        ti_m=os.path.getmtime(xle_f)
        m_ti=time.ctime(ti_m)
        t_obj=time.strptime(m_ti)
        xle_T_stamp=time.strftime("%Y-%m-%dT%H:%M:%S.000000Z",t_obj)


        # print(xle_file_size)

        md5_hash=hashlib.md5()
        file=open(csv_f,"rb")
        content=file.read()
        md5_hash.update(content)
        csv_digest=md5_hash.hexdigest()

        csv_file_size= os.path.getsize(csv_f)

        ti_m=os.path.getmtime(csv_f)
        m_ti=time.ctime(ti_m)
        t_obj=time.strptime(m_ti)
        csv_T_stamp=time.strftime("%Y-%m-%dT%H:%M:%S.000000Z",t_obj)

        #Need to be sure that this is in UTC time and NOT EST. I think it is getting the time based on the time from the local device, might beed to configure something on the cluster?
        current_time= str(datetime.now()).replace(" ","T")+("Z")
        # print(current_time)

        root= et.Element('GranuleMetaDataFile')

        for index,row in CollectionMetaData.iterrows():
            Column_heading_1 = et.SubElement(root,'Project')
            Column_heading_2 = et.SubElement(root, 'FileIdentifier')

        root_tags=et.SubElement(root,'CollectionMetaData')

        for index,row in CollectionMetaData.iterrows():
            Column_heading_3 = et.SubElement(root_tags, 'Campaign')
            Column_heading_4 = et.SubElement(root_tags, 'CampaignShortName')
            Column_heading_5 = et.SubElement(root_tags, 'ShortName')
            Column_heading_6 = et.SubElement(root_tags, 'InstrumentName')
            Column_heading_7 = et.SubElement(root_tags, 'Description')

            Column_heading_1.text = str(row['Project'])
            Column_heading_2.text = str(row['FileIdentifier'])
            Column_heading_3.text = str(row['Campaign'])
            Column_heading_4.text = str(row['CampaignShortName'])
            Column_heading_5.text = str(row['ShortName'])
            Column_heading_6.text = str(row['InstrumentName'])
            Column_heading_7.text = str(row['Description'])

        root_tags = et.SubElement(root, 'Extent')
        root_tags1 = et.SubElement(root_tags,'TemporalExtent')
        root_tags2 = et.SubElement(root_tags,'SpatialExtent')

        for index,row in CollectionMetaData.iterrows():
            Column_heading_8 = et.SubElement(root_tags1, 'StartDateTime')
            Column_heading_9 = et.SubElement(root_tags1, 'EndDateTime')
            Column_heading_10 = et.SubElement(root_tags2, 'WestBoundLongitude')
            Column_heading_11 = et.SubElement(root_tags2, 'EastBoundLongitude')
            Column_heading_12 = et.SubElement(root_tags2, 'SouthBoundLatitude')
            Column_heading_13 = et.SubElement(root_tags2, 'NorthBoundLatitude')

            Column_heading_8.text = str(row['StartDateTime'])
            Column_heading_9.text = str(row['EndDateTime'])
            Column_heading_10.text = str(row['WestBoundLongitude'])
            Column_heading_11.text = str(row['EastBoundLongitude'])
            Column_heading_12.text = str(row['SouthBoundLatitude'])
            Column_heading_13.text = str(row['NorthBoundLatitude'])

        root_tags = et.SubElement(root, 'DataGranuleMembers')

        for index,row in CollectionMetaData.iterrows():
            root_tags1= et.SubElement(root_tags,'DataGranuleMember')
            Column_heading_14 = et.SubElement(root_tags1, 'LocalGranuleID')
            Column_heading_15 = et.SubElement(root_tags1, 'ProductionDateTime')

            Column_heading_14.text = str(row['LocalGranuleID'])
            Column_heading_15.text = str(current_time)


        for index,row in DataGranuleMembers.iterrows():
            if str(".xle") in (row['LocalGranuleID']):
                root_tags1= et.SubElement(root_tags,'DataGranuleMember')
                Column_heading_16 = et.SubElement(root_tags1, 'LocalGranuleID')
                Column_heading_17= et.SubElement(root_tags1, 'ProductionDateTime')
                Column_heading_18 = et.SubElement(root_tags1, 'Checksum')
                Column_heading_19 = et.SubElement(root_tags1, 'CheckSumType')
                Column_heading_20 = et.SubElement(root_tags1, 'Size')

                Column_heading_16.text = str(row['LocalGranuleID'])
                Column_heading_17.text= str(xle_T_stamp)
                Column_heading_18.text = str(xle_digest)
                Column_heading_19.text= str(row['CheckSumType'])
                Column_heading_20.text = str(xle_file_size)
            else: 
                root_tags1= et.SubElement(root_tags,'DataGranuleMember')
                Column_heading_16 = et.SubElement(root_tags1, 'LocalGranuleID')
                Column_heading_17= et.SubElement(root_tags1, 'ProductionDateTime')
                Column_heading_18 = et.SubElement(root_tags1, 'Checksum')
                Column_heading_19 = et.SubElement(root_tags1, 'CheckSumType')
                Column_heading_20 = et.SubElement(root_tags1, 'Size')

                Column_heading_16.text = str(row['LocalGranuleID'])
                Column_heading_17.text= str(csv_T_stamp)
                Column_heading_18.text = str(csv_digest)
                Column_heading_19.text = str(row['CheckSumType'])
                Column_heading_20.text = str(csv_file_size)

        root_tags = et.SubElement(root, 'Additional')

        for index,row in CollectionMetaData.iterrows():
            Column_heading_20 = et.SubElement(root_tags, 'Personnel')
            Column_heading_21 = et.SubElement(root_tags, 'Contact')
            Column_heading_22 = et.SubElement(root_tags, 'PTModel')
            Column_heading_23 = et.SubElement(root_tags, 'PTSerial')
            Column_heading_24 = et.SubElement(root_tags, 'Latitude')
            Column_heading_25 = et.SubElement(root_tags, 'Longitude')
            Column_heading_26 = et.SubElement(root_tags, 'Method')
            Column_heading_27 = et.SubElement(root_tags, 'DateTimeinWater')
            Column_heading_28 = et.SubElement(root_tags, 'DateTimeoutofWater')
            Column_heading_29 = et.SubElement(root_tags, 'Install_GNSS_file')
            Column_heading_30 = et.SubElement(root_tags, 'Install_GNSS_start_datetime')
            Column_heading_31= et.SubElement(root_tags, 'Install_GNSS_end_datetime')
            Column_heading_32= et.SubElement(root_tags, 'Uninstall_GNSS_start_datetime')
            Column_heading_33= et.SubElement(root_tags, 'Uninstall_GNSS_end_datetime')

            Column_heading_20.text = str(row['Personnel'])
            Column_heading_21.text = str(row['Contact'])
            Column_heading_22.text = str(row['PTModel'])
            Column_heading_23.text = str(row['PTSerial'])
            Column_heading_24.text = str(row['Latitude'])
            Column_heading_25.text = str(row['Longitude'])
            Column_heading_26.text = str(row['Method'])
            Column_heading_27.text = str(row['DateTimeinWater'])
            Column_heading_28.text = str(row['DateTimeoutofWater'])
            Column_heading_29.text = str(row['Install_GNSS_file'])
            Column_heading_30.text = str(row['Install_GNSS_start_datetime'])
            Column_heading_31.text = str(row['Install_GNSS_end_datetime'])
            Column_heading_32.text = str(row['Uninstall_GNSS_start_datetime'])
            Column_heading_33.text = str(row['Uninstall_GNSS_end_datetime'])

        tree = et.ElementTree(root) 
        et.indent(tree, space="\t", level=0)
        tree.write(p_direc+'/'+os.path.basename(i.replace('.xlsx','.xml')), encoding="utf-8")
    if sensor == 'PT' and level== 'L2':
        f = i
        xle_f= i.replace('.xlsx', '.xle')
        csv_f= i.replace('.xlsx', '.csv')

        
#                 xlsx = []
#                 for i in os.listdir(unprocessed_spreadsheets):
#                         if i.endswith('.xlsx'):
#                             i_split = i.split('.')[0]
#                             list = xlsx.append(i_split)
#                         else:
#                             continue

#                 for i in range(len(xlsx)):

#                     f = direc+'/'+xlsx[i] +'.xlsx'
#                     xle_f= direc+'/'+xlsx[i] + '.xle'
#                     csv_f= direc+'/'+xlsx[i] + '.csv'

        project_prep = pd.read_excel(f, sheet_name='Project')
        projectdf= pd.DataFrame(project_prep)
        CollectionMetaData=flip_function(projectdf)
        DataGranuleMembers = pd.read_excel(f, sheet_name='DataGranuleMembers')


        md5_hash=hashlib.md5()
        file=open(xle_f,"rb")
        content=file.read()
        md5_hash.update(content)
        xle_digest=md5_hash.hexdigest()

        xle_file_size = os.path.getsize(xle_f)

        ti_m=os.path.getmtime(xle_f)
        m_ti=time.ctime(ti_m)
        t_obj=time.strptime(m_ti)
        xle_T_stamp=time.strftime("%Y-%m-%dT%H:%M:%S.000000Z",t_obj)

        # print(xle_file_size)

        md5_hash=hashlib.md5()
        file=open(csv_f,"rb")
        content=file.read()
        md5_hash.update(content)
        csv_digest=md5_hash.hexdigest()

        csv_file_size= os.path.getsize(csv_f)

        ti_m=os.path.getmtime(csv_f)
        m_ti=time.ctime(ti_m)
        t_obj=time.strptime(m_ti)
        csv_T_stamp=time.strftime("%Y-%m-%dT%H:%M:%S.000000Z",t_obj)

        current_time= str(datetime.now()).replace(" ","T")+("Z")

        root= et.Element('GranuleMetaDataFile')

        for index,row in CollectionMetaData.iterrows():
            Column_heading_1 = et.SubElement(root,'Project')
            Column_heading_2 = et.SubElement(root, 'FileIdentifier')

        root_tags=et.SubElement(root,'CollectionMetaData')

        for index,row in CollectionMetaData.iterrows():
            Column_heading_3 = et.SubElement(root_tags, 'Campaign')
            Column_heading_4 = et.SubElement(root_tags, 'CampaignShortName')
            Column_heading_5 = et.SubElement(root_tags, 'ShortName')
            Column_heading_6 = et.SubElement(root_tags, 'InstrumentName')
            Column_heading_7 = et.SubElement(root_tags, 'Description')

            Column_heading_1.text = str(row['Project'])
            Column_heading_2.text = str(row['FileIdentifier'])
            Column_heading_3.text = str(row['Campaign'])
            Column_heading_4.text = str(row['CampaignShortName'])
            Column_heading_5.text = str(row['ShortName'])
            Column_heading_6.text = str(row['InstrumentName'])
            Column_heading_7.text = str(row['Description'])

        root_tags = et.SubElement(root, 'Extent')
        root_tags1 = et.SubElement(root_tags,'TemporalExtent')
        root_tags2 = et.SubElement(root_tags,'SpatialExtent')

        for index,row in CollectionMetaData.iterrows():
            Column_heading_8 = et.SubElement(root_tags1, 'StartDateTime')
            Column_heading_9 = et.SubElement(root_tags1, 'EndDateTime')
            Column_heading_10 = et.SubElement(root_tags2, 'WestBoundLongitude')
            Column_heading_11 = et.SubElement(root_tags2, 'EastBoundLongitude')
            Column_heading_12 = et.SubElement(root_tags2, 'SouthBoundLatitude')
            Column_heading_13 = et.SubElement(root_tags2, 'NorthBoundLatitude')

            Column_heading_8.text = str(row['StartDateTime'])
            Column_heading_9.text = str(row['EndDateTime'])
            Column_heading_10.text = str(row['WestBoundLongitude'])
            Column_heading_11.text = str(row['EastBoundLongitude'])
            Column_heading_12.text = str(row['SouthBoundLatitude'])
            Column_heading_13.text = str(row['NorthBoundLatitude'])

        root_tags = et.SubElement(root, 'DataGranuleMembers')

        for index,row in CollectionMetaData.iterrows():
            root_tags1= et.SubElement(root_tags,'DataGranuleMember')
            Column_heading_14 = et.SubElement(root_tags1, 'LocalGranuleID')
            Column_heading_15 = et.SubElement(root_tags1, 'ProductionDateTime')

            Column_heading_14.text = str(row['LocalGranuleID'])
            Column_heading_15.text = str(current_time)

        for index,row in DataGranuleMembers.iterrows():
            if str(".xle") in (row['LocalGranuleID']):
                root_tags1= et.SubElement(root_tags,'DataGranuleMember')
                Column_heading_16 = et.SubElement(root_tags1, 'LocalGranuleID')
                Column_heading_17 = et.SubElement(root_tags1, 'ProductionDateTime')
                Column_heading_18 = et.SubElement(root_tags1, 'Checksum')
                Column_heading_19 = et.SubElement(root_tags1, 'CheckSumType')
                Column_heading_20 = et.SubElement(root_tags1, 'Size')

                Column_heading_16.text = str(row['LocalGranuleID'])
                Column_heading_17.text = str(xle_T_stamp)
                Column_heading_18.text = str(xle_digest)
                Column_heading_19.text = str(row['CheckSumType'])
                Column_heading_20.text = str(xle_file_size)

            else:
                Column_heading_16 = et.SubElement(root_tags1, 'LocalGranuleID')
                Column_heading_17 = et.SubElement(root_tags1, 'ProductionDateTime')
                Column_heading_18 = et.SubElement(root_tags1, 'Checksum')
                Column_heading_19 = et.SubElement(root_tags1, 'CheckSumType')
                Column_heading_20 = et.SubElement(root_tags1, 'Size')

                Column_heading_16.text = str(row['LocalGranuleID'])
                Column_heading_17.text = str(csv_T_stamp)
                Column_heading_18.text = str(csv_digest)
                Column_heading_19.text = str(row['CheckSumType'])
                Column_heading_20.text = str(csv_file_size)

        root_tags = et.SubElement(root, 'Additional')

        for index,row in CollectionMetaData.iterrows():
            Column_heading_21 = et.SubElement(root_tags, 'Personnel')
            Column_heading_22 = et.SubElement(root_tags, 'Contact')
            Column_heading_23 = et.SubElement(root_tags, 'PTModel')
            Column_heading_24 = et.SubElement(root_tags, 'PTSerial')
            Column_heading_25 = et.SubElement(root_tags, 'Latitude')
            Column_heading_26 = et.SubElement(root_tags, 'Longitide')
            Column_heading_27 = et.SubElement(root_tags, 'Method')
            Column_heading_28 = et.SubElement(root_tags, 'DateTimeinWater')
            Column_heading_29 = et.SubElement(root_tags, 'DateTimeoutofWater')
            Column_heading_30 = et.SubElement(root_tags, 'Install_GNSS_file')
            Column_heading_31 = et.SubElement(root_tags, 'Install_GNSS_start_datetime')
            Column_heading_32 = et.SubElement(root_tags, 'Install_GNSS_end_datetime')
            Column_heading_33 = et.SubElement(root_tags, 'Reference_GNSS_WSE_m')
            Column_heading_34 = et.SubElement(root_tags, 'Reference_DateTime_WSE')
            Column_heading_35 = et.SubElement(root_tags, 'DateTimeoutofWater')
            Column_heading_36 = et.SubElement(root_tags, 'Uninstall_GNSS_file')
            Column_heading_37 = et.SubElement(root_tags, 'Uninstall_GNSS_start_datetime')
            Column_heading_38 = et.SubElement(root_tags, 'Uninstall_GNSS_end_datetime')
            Column_heading_39 = et.SubElement(root_tags, 'Uninstall_Check_GNSS_WSE')
            Column_heading_40 = et.SubElement(root_tags, 'Uninstall_Check_DateTime_WSE')
            Column_heading_41 = et.SubElement(root_tags,'Barologger_File')
            Column_heading_42 = et.SubElement(root_tags, 'Density_Correction_Method')

            Column_heading_21.text = str(row['Personnel'])
            Column_heading_22.text = str(row['Contact'])
            Column_heading_23.text = str(row['PTModel'])
            Column_heading_24.text = str(row['PTSerial'])
            Column_heading_25.text = str(row['Latitude'])
            Column_heading_26.text = str(row['Longitude'])
            Column_heading_27.text = str(row['Method'])
            Column_heading_28.text = str(row['DateTimeinWater'])
            Column_heading_29.text = str(row['DateTimeoutofWater'])
            Column_heading_30.text = str(row['Install_GNSS_file'])
            Column_heading_31.text = str(row['Install_GNSS_start_datetime'])
            Column_heading_32.text = str(row['Install_GNSS_end_datetime'])
            Column_heading_33.text = str(row['Reference_GNSS_WSE_m'])
            Column_heading_34.text = str(row['Reference_DateTime_WSE'])
            Column_heading_35.text = str(row['DateTimeoutofWater'])
            Column_heading_36.text = str(row['Uninstall_GNSS_file'])
            Column_heading_37.text = str(row['Uninstall_GNSS_start_datetime'])
            Column_heading_38.text = str(row['Uninstall_GNSS_end_datetime'])
            Column_heading_39.text = str(row['Uninstall_Check_GNSS_WSE'])
            Column_heading_40.text = str(row['Uninstall_Check_DateTime_WSE'])
            Column_heading_41.text = str(row['Barologger_File'])
            Column_heading_42.text = str(row['Density_Correction_Method'])

        tree = et.ElementTree(root) 
        et.indent(tree, space="\t", level=0)
        tree.write(p_direc+'/'+os.path.basename(i.replace('.xlsx','.xml')), encoding="utf-8")
#                 else:
#                     print('Error: There is something wrong here')




AttributeError: 'str' object has no attribute 'isfile'